<a href="https://colab.research.google.com/github/ojcharles/snippets/blob/main/nnet_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple MNIST NN from scratch (numpy, no TF/Keras)

## Simple MNIST NN from scratch , using python and numpy

In [42]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import datasets

In [49]:
digits = datasets.load_digits()
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))
labels = digits.target
data = np.insert(data, 0, labels, axis=1) # index 0 is label, others are ordered pixel values from 0-255 in value
print(data)
print(data.shape)

[[ 0.  0.  0. ...  0.  0.  0.]
 [ 1.  0.  0. ... 10.  0.  0.]
 [ 2.  0.  0. ... 16.  9.  0.]
 ...
 [ 8.  0.  0. ...  6.  0.  0.]
 [ 9.  0.  0. ... 12.  0.  0.]
 [ 8.  0.  0. ... 12.  1.  0.]]
(1797, 65)


In [67]:
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0].astype("int")
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

Y_train

array([0, 6, 3, 5, 2, 0, 0, 0, 5, 4, 9, 8, 0, 4, 1, 7, 0, 4, 4, 0, 5, 1,
       8, 8, 0, 5, 0, 7, 8, 9, 7, 6, 4, 9, 7, 9, 6, 7, 3, 7, 1, 0, 7, 3,
       1, 7, 3, 7, 6, 2, 8, 9, 8, 0, 9, 3, 1, 5, 0, 0, 7, 0, 6, 0, 9, 2,
       0, 8, 1, 3, 7, 1, 5, 1, 7, 4, 5, 7, 5, 3, 3, 3, 0, 1, 5, 7, 6, 2,
       6, 7, 8, 5, 5, 4, 6, 0, 1, 4, 4, 4, 9, 6, 2, 2, 2, 9, 2, 0, 5, 1,
       9, 9, 9, 5, 0, 6, 7, 1, 8, 8, 9, 0, 0, 9, 9, 8, 7, 4, 7, 8, 9, 4,
       5, 9, 5, 8, 8, 6, 1, 1, 4, 8, 8, 1, 7, 0, 1, 6, 4, 6, 9, 1, 2, 4,
       0, 7, 6, 7, 5, 1, 1, 6, 4, 7, 1, 5, 8, 2, 0, 2, 3, 1, 2, 5, 3, 8,
       6, 0, 0, 2, 6, 7, 3, 7, 7, 0, 3, 9, 8, 1, 6, 4, 3, 7, 3, 3, 8, 6,
       7, 0, 8, 7, 3, 3, 4, 0, 1, 3, 2, 5, 5, 5, 3, 4, 8, 8, 1, 5, 8, 5,
       4, 5, 2, 4, 1, 8, 3, 7, 8, 3, 6, 8, 0, 5, 0, 6, 0, 6, 1, 3, 4, 3,
       2, 4, 9, 3, 9, 8, 9, 3, 7, 3, 6, 5, 3, 0, 0, 6, 3, 0, 5, 8, 5, 1,
       1, 1, 4, 8, 4, 8, 0, 8, 5, 9, 6, 6, 4, 7, 6, 3, 9, 2, 5, 3, 0, 9,
       4, 6, 4, 9, 9, 8, 2, 7, 7, 2, 3, 2, 4, 6, 7,

In [68]:
X_train.shape

(64, 797)

Building a nnet from scratch

In [75]:
def init_params():
    W1 = np.random.rand(10, 64) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [76]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

Training!

In [77]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 